In [2]:
import requests
import requests_cache
import lxml.html as lx
import re
from selenium import webdriver
import time
import random
import nltk

Testing to find format (for project report)

In [13]:
response= requests.get("https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc")
html_tree = lx.fromstring(response.text) #not necessairly working might need to use api

reviews = html_tree.xpath("//div[@id='reviews']//ul[@class = ' list__09f24__ynIEd']/li//div[@class = ' y-css-1pnalxe']/p/span/text()")

In [10]:
nextbuttons = html_tree.xpath("//span[@class = ' y-css-yrt0i5']//a[@class = 'pagination-link-component__09f24__JRiQO y-css-14k1rqv']/@href")
nextbuttons 

NameError: name 'html_tree' is not defined

In [18]:
html_tree.xpath("(//a[div[@class='undefined y-css-1wz9c5l']])[last()]/div/@aria-label")


['Page: 9']

In [9]:
reviewlist = []
counter = 0 

while counter < (len(nextbuttons)):
    response = requests.get("https://www.yelp.com/biz/muni-bart-station-montgomery-san-francisco?sort_by=date_desc")
    html_tree = lx.fromstring(response.text)
    reviews = html_tree.xpath("//div[@id='reviews']//ul[@class = ' list__09f24__ynIEd']/li//div[@class = ' y-css-1pnalxe']/p/span/text()")
    reviewlist.extend(reviews)
    nextbuttons = html_tree.xpath("(//span[@class = ' y-css-yrt0i5']//a[@class = 'pagination-link-component__09f24__JRiQO y-css-14k1rqv'])/@href")
    for link in nextbuttons:
        if response.status_code != 200:
            print(f"Failed to fetch page: {link}")
            break
        else:
                response = requests.get(link)
                html_tree = lx.fromstring(response.text)
                reviews = html_tree.xpath("//div[@id='reviews']//ul[@class = ' list__09f24__ynIEd']/li//div[@class = ' y-css-1pnalxe']/p/span/text()")
                reviewlist.extend(reviews)
                counter+=1
                sleep_time = random.uniform(2, 5)  # Random delay to reduce likelihood of detection
                time.sleep(sleep_time)

NameError: name 'nextbuttons' is not defined

In [16]:
len(reviewlist)

151

Clean Code

In [46]:
def GetPages(url):
    links_to_visit = [url]  # Initialize with the parent URL
    processed_links = set()  # Track links that have already been processed
    unique_start_urls = {}  # Track URLs with unique `start` values

    while links_to_visit:
        link = links_to_visit.pop()  # Lifo

        if link in processed_links:  # Skip if already processed
            continue
        processed_links.add(link)  # Mark as processed

        try:
            response = requests.get(link, timeout=10)
            if response.status_code != 200:
                time.sleep(random.uniform(2, 10))
                response = requests.get(link)
                if response.status_code != 200:
                    time.sleep(random.uniform(10, 20))
                    response = requests.get(link)
            
            html_tree = lx.fromstring(response.text)
            
            # Extract pagination links
            new_links = html_tree.xpath("//a[contains(@class, 'pagination-link-component')]/@href")

            # Deduplicate links based on `start` parameter
            for new_link in new_links:
                start_value = new_link.split("start=")[1].split("&")[0] if "start=" in new_link else "0"
                if start_value not in unique_start_urls:  # Check if `start` value is unique
                    unique_start_urls[start_value] = new_link
                    links_to_visit.append(new_link)  # Add unique link to visit next

        except Exception as e:
            continue

        time.sleep(random.uniform(1, 3))  # Pause between requests

    return list(unique_start_urls.values())  # Return all unique URLs based on `start` values


In [34]:
def GetReviews(url):
    reviewlist = []  # List to store reviews
    
    # Request the page
    response = requests.get(url)
    if response.status_code != 200:
        time.sleep(random.uniform(2, 10))
        response = requests.get(url)
        if response.status_code != 200:
            time.sleep(random.uniform(10, 20))
            response = requests.get(url)
            if response.status_code != 200:
                return reviewlist  # Return an empty list if the request fails

    # Parse the page content
    html_tree = lx.fromstring(response.text)
    
    # Extract reviews using XPath
    reviews = html_tree.xpath("//div[@id='reviews']//ul/li//p/span/text()")
    reviewlist.extend(reviews)
    
    # Random delay to avoid detection
    time.sleep(random.uniform(2, 5))

    return reviewlist

In [35]:
parenturlist = ["https://www.yelp.com/biz/muni-bart-station-embarcadero-san-francisco?sort_by=date_desc",
          "https://www.yelp.com/biz/muni-bart-station-montgomery-san-francisco?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-muni-powell-street-station-san-francisco?sort_by=date_desc",
          "https://www.yelp.com/biz/muni-bart-station-civic-center-san-francisco?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-24th-st-mission-station-san-francisco-2?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-glen-park-station-san-francisco?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-balboa-park-station-san-francisco?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-west-oakland-station-oakland?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-12th-street-station-oakland?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-19th-street-station-oakland?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-macarthur-station-oakland?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-rockridge-station-oakland-2?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-fruitvale-station-oakland?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-coliseum-station-oakland?sort_by=date_desc",
          "https://www.yelp.com/biz/bart-lake-merritt-station-oakland?sort_by=date_desc",
         "https://www.yelp.com/biz/bart-oakland-international-airport-station-oakland-2?sort_by=date_desc"]

In [47]:
GetPages("https://www.yelp.com/biz/bart-oakland-international-airport-station-oakland-2?sort_by=date_desc") #test

[]

In [30]:
filtered_reviews = {
    url: [review for review in review_list if len(review.split()) > 2] #to make sure paragraphed reviews are longer than two words
    for url, review_list in reviewdict.items()
}
filtered_reviews

{'https://www.yelp.com/biz/muni-bart-station-embarcadero-san-francisco?sort_by=date_desc': ["This is one of the busiest stops in the BART system.  It's cleaner than most.  It's also full of professionals and has a lower number of tourists.  If you want tourists, go to Powell St. Station.",
  'I love the design of the station.  You can look up and see the Muni Metro train rumbling above.  And you can hear the Muni automated announcements too.',
  "What's really annoying is the constantly broken escalators.  3 months to fix one of them the last time.    Plus, the station can be cleaned.  It's a bit grimey.   ",
  "Still, it's convenient and centrally located.  It feeds the FiDi and the SoMa and the Waterfront, where you can catch a bus, a ferry, a cable car, a historic trolley, or a taxi.",
  "I'm Okay with this Bart Stop, it's got an Internet signal and it's usually clean. If it's rush hour in the afternoon, getting on here (the last stop before going thru the tube) there's usually no s

In [22]:
total_values = sum(len(value) if isinstance(value, list) else 0 for value in filtered_reviews.values())
print("There are " + str(total_values) + " reviews") #400 reviews filtered out

There are 7819 reviews


In [23]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

In [24]:
sid = SentimentIntensityAnalyzer()

def average_compound_score(reviews):
    """calculates the average compound sentiment score for a list of reviews."""
    compound_scores = []
    for sentence in reviews:
        ss = sid.polarity_scores(sentence)
        compound_scores.append(ss['compound'])
    return sum(compound_scores) / len(compound_scores) if compound_scores else 0

for key, reviews in filtered_reviews.items():  #goes through every review, uses nltk's VADER logic to compute and output sentimet score [-1,1]
    print(f"\nReviews for {key}:")
    for i, sentence in enumerate(reviews, start=1):
        print(f"  Review {i}: {sentence}")
        ss = sid.polarity_scores(sentence)
        for k in sorted(ss):
            print(f'    {k}: {ss[k]:.2f}', end=', ')
        print()  # newline after scores


Reviews for https://www.yelp.com/biz/muni-bart-station-embarcadero-san-francisco?sort_by=date_desc:
  Review 1: This station has always had a problem with the homeless and it feels like the issues at this station has got worse over the years and both muni and Bart aren't doing much to solve it or to make it any better. If you're going through this station be careful.
    compound: 0.25,     neg: 0.10,     neu: 0.75,     pos: 0.15, 
  Review 2: Even though this is one of my favorite stations, I have to give it 1 star because I saw at least two people evade the toll on the way in. They did not have to pay today so I can't give it any more than one. Be better BART!
    compound: 0.76,     neg: 0.00,     neu: 0.85,     pos: 0.15, 
  Review 3: This is the first station coming into San Francisco from the East Bay, and probably the one I frequent the most now that I no longer work in The City.  
    compound: -0.35,     neg: 0.08,     neu: 0.92,     pos: 0.00, 
  Review 4: It is safe, well p

In [25]:
average_scores_dict = {}

# Iterate through the reviewdict and calculate average compound score for each station
for key, reviews in reviewdict.items():
    avg_compound = average_compound_score(reviews)
    average_scores_dict[key] = avg_compound

average_scores_dict

{'https://www.yelp.com/biz/muni-bart-station-embarcadero-san-francisco?sort_by=date_desc': 0.08704808823529406,
 'https://www.yelp.com/biz/muni-bart-station-montgomery-san-francisco?sort_by=date_desc': 0.17420888554216854,
 'https://www.yelp.com/biz/bart-muni-powell-street-station-san-francisco?sort_by=date_desc': 0.14884712765957436,
 'https://www.yelp.com/biz/muni-bart-station-civic-center-san-francisco?sort_by=date_desc': 0.09249411042944783,
 'https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc': 0.05925118279569891,
 'https://www.yelp.com/biz/bart-24th-st-mission-station-san-francisco-2?sort_by=date_desc': 0.08819187279151941,
 'https://www.yelp.com/biz/bart-glen-park-station-san-francisco?sort_by=date_desc': 0.3414623556581987,
 'https://www.yelp.com/biz/bart-balboa-park-station-san-francisco?sort_by=date_desc': 0.13342076023391808,
 'https://www.yelp.com/biz/bart-west-oakland-station-oakland?sort_by=date_desc': 0.19625679513184582,
 'https://www.yelp.co

Best to Worst: 
Glen Park
Rockridge
West Oakland
Montgomery
Lake Merrit 
Macarthur
Powell Street
Coliseum
Fruitvale
Balboa Park
Civic Center
24th Street
Embarcadero
12th Street
19th Street
16th Street

In [26]:
import pandas as pd

In [37]:
sentimentdf = pd.DataFrame.from_dict(average_scores_dict, orient='index', dtype=None, columns= None)
sentimentdf.rename(columns={0: 'Sentiment Score'}, inplace=True) 
sentimentdf['ranking'] = sentimentdf['Sentiment Score'].rank(ascending=False, method='min')
sentimentdf

,Sentiment Score,ranking
https://www.yelp.com/biz/muni-bart-station-embarcadero-san-francisco?sort_by=date_desc,0.087048,13.0
https://www.yelp.com/biz/muni-bart-station-montgomery-san-francisco?sort_by=date_desc,0.174209,4.0
https://www.yelp.com/biz/bart-muni-powell-street-station-san-francisco?sort_by=date_desc,0.148847,7.0
https://www.yelp.com/biz/muni-bart-station-civic-center-san-francisco?sort_by=date_desc,0.092494,11.0
https://www.yelp.com/biz/bart-16th-st-mission-san-francisco?sort_by=date_desc,0.059251,16.0
https://www.yelp.com/biz/bart-24th-st-mission-station-san-francisco-2?sort_by=date_desc,0.088192,12.0
https://www.yelp.com/biz/bart-glen-park-station-san-francisco?sort_by=date_desc,0.341462,1.0
https://www.yelp.com/biz/bart-balboa-park-station-san-francisco?sort_by=date_desc,0.133421,10.0
https://www.yelp.com/biz/bart-west-oakland-station-oakland?sort_by=date_desc,0.196257,3.0
https://www.yelp.com/biz/bart-12th-street-station-oakland?sort_by=date_desc,0.083885,14.0
